In [1]:
#!/usr/bin/env python3

import os, time, sys
sys.path.append('../')
import numpy as np
import pyhepmc
import random
import logging
import configparser
import subprocess
import multiprocessing
import itertools
import progressbar as P
from helper import getModelDict
import getEffsFromROOT
import getEffsFromROOTv2
delphesDir = os.path.abspath("../DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

c = 3e8

FORMAT = '%(levelname)s: %(message)s at %(asctime)s'
logging.basicConfig(format=FORMAT,datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()   

Welcome to JupyROOT 6.30/06


In [2]:
inputFile = '../pp2chi0chi0_minimalH_scan_jets/Events/run_01/ddmH_mS_2000_m1_979_dm_90_delphes_events.root'
tauList = [4.]
llps = [4000023]
invisibles = [12,14,16,4000022]
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")

In [3]:
tree.GetEntry(0)

23372

In [4]:
evt_effs,evt_cutFlow = getEffsFromROOT.getEffFor(tree,tauList,llps=llps,invisibles=invisibles)
print(evt_effs)
print(evt_cutFlow)

[0.]
{'All': array([1.]), 'Eta cut': array([1.]), 'Energy theshold': array([1.]), 'Delta Phi': array([0.]), 'Decay Position': array([0.]), 'Decay Time': array([0.])}


In [5]:
evt_effs,evt_cutFlow = getEffsFromROOTv2.getEffFor(tree,tauList,llps=llps,invisibles=invisibles)
print(evt_effs)
print(evt_cutFlow)

[0.]
{'All': array([1.]), 'Eta cut': array([1.]), 'Energy theshold': array([0.]), 'Delta Phi': array([0.]), 'Decay Position': array([0.]), 'Decay Time': array([0.])}


In [6]:
eventDict = getEffsFromROOT.getDataFrom(tree,llps=llps,invisibles=invisibles)
print('LLP1:')
for key in eventDict[0]:
    if key == 'cluster':
        print('  ',key,eventDict[0][key].momentum.pt())
    else:
        print('  ',key,eventDict[0][key])
print('LLP2:')
for key in eventDict[1]:
    if key == 'cluster':
        print('  ',key,eventDict[1][key].momentum.pt())
    else:
        print('  ',key,eventDict[1][key])


LLP1:
   visible GenParticle(FourVector(34.8, -32.1, 116, 144), pid=0, status=0)
   invisible GenParticle(FourVector(-135, 5.43, 1.46e+03, 1.71e+03), pid=0, status=0)
   cluster 49.00502395629883
   parent GenParticle(FourVector(-100, -26.7, 1.57e+03, 1.85e+03), pid=4000023, status=0)
   visiblePDGs [5, -5]
   invisiblePDGs [-4000022]
LLP2:
   visible GenParticle(FourVector(26.8, 23.3, 81.4, 116), pid=1, status=0)
   invisible GenParticle(FourVector(852, 458, 760, 1.52e+03), pid=1, status=0)
   cluster 28.322751775967543
   parent GenParticle(FourVector(879, 481, 842, 1.63e+03), pid=-4000023, status=0)
   visiblePDGs [5, -5]
   invisiblePDGs [-4000022]


In [7]:
# Extract necessary data from event
eventDict_ontime = getEffsFromROOTv2.getDataFrom(tree.llpParticlesOnTime,
                                tree.llpDirectDaughtersOnTime,
                                llp_PDGs=llps,invisible_PDGs=invisibles)
eventDict_delayed = getEffsFromROOTv2.getDataFrom(tree.llpParticlesDelayed,
                                tree.llpDirectDaughtersDelayed,
                                llp_PDGs=llps,invisible_PDGs=invisibles)

# Get on-time and delayed jets
jets_ontime = getEffsFromROOTv2.getJetsFrom(tree.GenJetOnTime)
jets_delayed = getEffsFromROOTv2.getJetsFrom(tree.GenJetDelayed)

print('LLP on-time:')
for key in eventDict_ontime[0]:
    print('  ',key,eventDict_ontime[0][key])
for j in jets_ontime:
    print('  *cluster* = ',j.momentum.pt())
print('LLP delayed:')
for key in eventDict_delayed[0]:
    print('  ',key,eventDict_delayed[0][key])
for j in jets_delayed:
    print('  *cluster* = ',j.momentum.pt())

LLP on-time:
   visible GenParticle(FourVector(34.8, -32.1, 116, 144), pid=0, status=0)
   invisible GenParticle(FourVector(-135, 5.43, 1.46e+03, 1.71e+03), pid=0, status=0)
   parent GenParticle(FourVector(-100, -26.7, 1.57e+03, 1.85e+03), pid=4000023, status=0)
   visiblePDGs [5, -5]
   invisiblePDGs [4000022]
  *cluster* =  899.0795288085936
  *cluster* =  49.00502395629883
  *cluster* =  32.47551345825195
  *cluster* =  20.471296310424805
LLP delayed:
   visible GenParticle(FourVector(26.8, 23.3, 81.4, 116), pid=0, status=0)
   invisible GenParticle(FourVector(852, 458, 760, 1.52e+03), pid=0, status=0)
   parent GenParticle(FourVector(879, 481, 842, 1.63e+03), pid=-4000023, status=0)
   visiblePDGs [5, -5]
   invisiblePDGs [-4000022]
  *cluster* =  43.50146484375
  *cluster* =  26.193784713745117
